## <b>DATA</b>

In [1]:
from ucimlrepo import fetch_ucirepo 
from joblib import dump, load
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
# fetch dataset 
wine_quality = fetch_ucirepo(id=186) 
  
# data (as pandas dataframes) 
X = wine_quality.data.features 
y = wine_quality.data.targets 

In [3]:
# sepsis features
X.head(5)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


In [4]:
# sepsis targets
y.head(5)

,quality
0,5
1,5
2,5
3,6
4,5


In [5]:
y['quality'].value_counts()

quality
6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: count, dtype: int64

In [6]:
# make binary and ravel

y_encode = y.copy()
y_encode['quality'] = (y['quality'] >= 6).astype(float)

y_n = np.array(y_encode).ravel()

y_n

array([0., 0., 0., ..., 1., 1., 1.])

In [7]:
# normalizing
scaler = StandardScaler()
X_n = scaler.fit_transform(X)

In [8]:
X_n

array([[ 0.14247327,  2.18883292, -2.19283252, ...,  1.81308951,
         0.19309677, -0.91546416],
       [ 0.45103572,  3.28223494, -2.19283252, ..., -0.11507303,
         0.99957862, -0.58006813],
       [ 0.45103572,  2.55330026, -1.91755268, ...,  0.25811972,
         0.79795816, -0.58006813],
       ...,
       [-0.55179227, -0.6054167 , -0.88525328, ..., -1.42124765,
        -0.47897144, -0.91546416],
       [-1.32319841, -0.30169391, -0.12823371, ...,  0.75571005,
        -1.016626  ,  1.9354021 ],
       [-0.93749534, -0.78765037,  0.42232597, ...,  0.25811972,
        -1.41986693,  1.09691202]])

## <b>Gradient Boosting (XGBoost)</b>


In [9]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE

In [10]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X_n, y_n, test_size=0.2, random_state=42, shuffle=True)

# experiment with class weights or SMOTE
# SMOTE - oversample minority class
smote = SMOTE(random_state=42, k_neighbors=1)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [11]:
#base model
xgb_base = XGBClassifier(eval_metric='logloss',
                         objective='binary:logistic',
                         reg_alpha = 0.5,
                         reg_lambda=5,
                         random_state=42)

# tune hyperparameters with gridsearch
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3,7, 10]
}

grid_search = GridSearchCV(
    estimator= xgb_base,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=5,
    verbose=1
) 

grid_search.fit(X_train_smote, y_train_smote)

print("Best Parameters:", grid_search.best_params_)
print("Best ROC-AUC:", grid_search.best_score_)

#consider class-specific thresholds
#precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba[:, class_index])
#optimal_threshold = thresholds[np.argmax(precision * recall)]

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters: {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 200}
Best ROC-AUC: 0.9330686240105287


In [12]:
# train on 3 splits
for test_size in [0.2,0.5, 0.8]:
    X_train, X_test, y_train, y_test = train_test_split(X_n, y_n, test_size=test_size,shuffle=True, random_state=42)
    
    # create model with optimal params
    xgb_opt = XGBClassifier(
        eval_metric='mlogloss',
        **grid_search.best_params_,
        random_state=42
    )
    
    xgb_opt.fit(X_train, y_train)
    
    y_pred_proba = xgb_opt.predict_proba(X_test)[:,1]
    y_pred_class = xgb_opt.predict(X_test)
    
    print("Train="+str(round(1-test_size,2)))
    print(classification_report(y_test, y_pred_class,zero_division=0))
    
    # ROC-AUC for multiclass MODIFY TMR
    try:
        roc_auc = roc_auc_score(y_test, y_pred_proba)  
        print(f"ROC-AUC: {roc_auc:.4f}")
    except ValueError as e:
        print(f"Could not calculate ROC-AUC: {e}")
        
    print(" ")
    print(" ")


Train=0.8
              precision    recall  f1-score   support

         0.0       0.73      0.72      0.72       451
         1.0       0.85      0.86      0.86       849

    accuracy                           0.81      1300
   macro avg       0.79      0.79      0.79      1300
weighted avg       0.81      0.81      0.81      1300

ROC-AUC: 0.8667
 
 
Train=0.5
              precision    recall  f1-score   support

         0.0       0.71      0.72      0.72      1157
         1.0       0.84      0.84      0.84      2092

    accuracy                           0.80      3249
   macro avg       0.78      0.78      0.78      3249
weighted avg       0.80      0.80      0.80      3249

ROC-AUC: 0.8600
 
 
Train=0.2
              precision    recall  f1-score   support

         0.0       0.67      0.68      0.67      1900
         1.0       0.81      0.81      0.81      3298

    accuracy                           0.76      5198
   macro avg       0.74      0.74      0.74      5198
weig

In [13]:
# save the model

dump(xgb_opt, "trained/xgb_opt_WINE.joblib")

['trained/xgb_opt_WINE.joblib']

## <b>NEURALNET</b>

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Input
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score,classification_report, roc_auc_score, roc_curve
from sklearn.preprocessing import label_binarize

In [15]:
# Split 80/20 train/test
X_normed_nn = scaler.fit_transform(X_n)
X_normed_nn

# splitting NORMED data
X_train, X_test, y_train, y_test = train_test_split(X_normed_nn, y_n, test_size=0.2, random_state=42,shuffle=True)

# Oversample the minority classes
smote = SMOTE(random_state=42, k_neighbors=1)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [16]:
def create_nn(learning_rate=0.001):
    model = Sequential([Input(shape=(11,)),
                        Dense(64, activation="relu"),
                        Dense(32, activation="relu"), 
                        Dense(1, activation="sigmoid")]) # output layer
    
    model.compile(optimizer = Adam(learning_rate=learning_rate),
                  loss = "binary_crossentropy",
                  metrics = ["accuracy"])
                  
    return model

In [17]:
# Keras wrapper -> use gridsearchcv with neuralnet

nn = KerasClassifier(model=create_nn)

In [18]:
# Gridsearchcv

param_grid = {
    'model__learning_rate': [0.001, 0.01],
    'batch_size': [64, 128, 256]
}

grid_search = GridSearchCV(
    estimator=nn,
    param_grid=param_grid,
    scoring="accuracy",
    cv=5,
    n_jobs=4     #parallel = 4 cores
) 

# early stop
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

grid_search.fit(
    X_train_smote,
    y_train_smote,
    epochs=200,
    callbacks=[early_stop]
)

print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

Epoch 1/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6990 - loss: 0.5660   
Epoch 2/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 993us/step - accuracy: 0.7706 - loss: 0.4973
Epoch 3/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 949us/step - accuracy: 0.7732 - loss: 0.4929
Epoch 4/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 977us/step - accuracy: 0.7815 - loss: 0.4714
Epoch 5/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 973us/step - accuracy: 0.7766 - loss: 0.4717
Epoch 6/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 955us/step - accuracy: 0.7870 - loss: 0.4590
Epoch 7/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 935us/step - accuracy: 0.7920 - loss: 0.4517
Epoch 8/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 933us/step - accuracy: 0.7924 - loss: 0.4446
Epoch 9/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 962us/step - accuracy: 0.8036 - loss: 0.4245
Epoch 10/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step - accuracy: 0.8129 - loss: 0.4163
Epoch 11/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 957us/step - accuracy: 0.8123 - loss: 0.4195
Epoch 12/200
51/51 ━━━━━━━━━━━━━━━━━━━━ 

In [19]:

# train on 3 splits
for test_size in [0.2,0.5, 0.8]:
    X_train, X_test, y_train, y_test = train_test_split(X_normed_nn, y_n, test_size=test_size, random_state=42,shuffle=True)
    
    # create model with optimal learning rate
    nn_opt = create_nn(
        learning_rate=grid_search.best_params_['model__learning_rate']
    )
    
    nn_opt.fit(X_train, y_train,
               batch_size=grid_search.best_params_['batch_size'],
               epochs=200,
               validation_split=0.2,
               callbacks=[early_stop])

    y_pred_proba = nn_opt.predict(X_test)
    y_pred_class = (y_pred_proba >= 0.5).astype(float)
      
    print("Train="+str(round(1-test_size)))
    print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba, multi_class='ovr'):.4f}")
    print(" ")
    print(" ")

           

Epoch 1/200
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6323 - loss: 0.6239 - val_accuracy: 0.7683 - val_loss: 0.4786
Epoch 2/200
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7536 - loss: 0.5017 - val_accuracy: 0.7635 - val_loss: 0.4767
Epoch 3/200
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7608 - loss: 0.4934 - val_accuracy: 0.7875 - val_loss: 0.4551
Epoch 4/200
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7808 - loss: 0.4743 - val_accuracy: 0.7942 - val_loss: 0.4589
Epoch 5/200
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7738 - loss: 0.4692 - val_accuracy: 0.7788 - val_loss: 0.4619
Epoch 6/200
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7786 - loss: 0.4670 - val_accuracy: 0.7798 - val_loss: 0.4671
Epoch 7/200
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7743 - loss: 0.4694 - val_accuracy: 0.7779 - val_loss: 0.4547
Epoch 8/200
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7896 - loss: 0.4567 - val_accuracy: 0.7913 - v

In [20]:
# save the model

dump(nn_opt, "trained/nn_opt_WINE.joblib")

['trained/nn_opt_WINE.joblib']

## <b>RandomForest</b>

In [21]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE

In [22]:
# Split 80/20 train/test
X_train, X_test, y_train, y_test = train_test_split(X_n, y_n, test_size=0.2, random_state=42,shuffle=True)

# Oversample the minority classes
smote = SMOTE(random_state=42,  k_neighbors=1)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [23]:
# gridsearchcv
param_grid = {
    'n_estimators': [50, 100, 200], #100 200
    'max_depth': [10, 20],
    'min_samples_split': [2, 5]
}

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42), 
    param_grid, 
    cv=3, 
    scoring='roc_auc',
    verbose=1,
    n_jobs=4
)

grid_search.fit(X_train_smote, y_train_smote)
print("Best Parameters:", grid_search.best_params_)
print("Best ROC-AUC:", grid_search.best_score_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best Parameters: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}
Best ROC-AUC: 0.9369413206459054


In [24]:
# Check most 'influential' feature to predict the target
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([0.06392109, 0.12212682, 0.07227384, 0.07204164, 0.08186791,
       0.0788032 , 0.07932974, 0.10037574, 0.06726297, 0.07764814,
       0.18434892])

In [25]:
#Train model with best parameters on 3 different splits

for test_size in [0.2,0.5, 0.8]:
    X_train, X_test, y_train, y_test = train_test_split(X_n, y_n, test_size=test_size, random_state=42,shuffle=True)    
    
    rf_opt = RandomForestClassifier(**grid_search.best_params_, random_state=42)
    
    rf_opt.fit(X_train, y_train)
    
    y_pred = rf_opt.predict(X_test)
    y_pred_proba = rf_opt.predict_proba(X_test)[:,1]
    
    print("Train="+str(round(1-test_size,2)))
    print(classification_report(y_test, y_pred, zero_division=0))
    print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")
    print(" ")
    print(" ")

Train=0.8
              precision    recall  f1-score   support

         0.0       0.75      0.73      0.74       451
         1.0       0.86      0.87      0.86       849

    accuracy                           0.82      1300
   macro avg       0.80      0.80      0.80      1300
weighted avg       0.82      0.82      0.82      1300

ROC-AUC: 0.8896
 
 
Train=0.5
              precision    recall  f1-score   support

         0.0       0.73      0.73      0.73      1157
         1.0       0.85      0.85      0.85      2092

    accuracy                           0.81      3249
   macro avg       0.79      0.79      0.79      3249
weighted avg       0.81      0.81      0.81      3249

ROC-AUC: 0.8792
 
 
Train=0.2
              precision    recall  f1-score   support

         0.0       0.69      0.66      0.67      1900
         1.0       0.81      0.83      0.82      3298

    accuracy                           0.77      5198
   macro avg       0.75      0.74      0.75      5198
weig

In [26]:
# save the model

dump(rf_opt, "trained/rf_opt_WINE.joblib")

['trained/rf_opt_WINE.joblib']

## <b>SGDClassifier (SVM approximation)</b>

In [41]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, classification_report


In [42]:
# Split 80/20 train/test

# splitting NORMED data
X_train, X_test, y_train, y_test = train_test_split(X_normed_nn, y_n, test_size=0.2, random_state=42,shuffle=True)

# Oversample the minority class
smote = SMOTE(random_state=42,k_neighbors=1)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [43]:
# base model
sgd = SGDClassifier(
    learning_rate='optimal',
    penalty='elasticnet',
    max_iter=200,
    random_state=42
)

#gridsearch
param_grid = {
    'alpha' : [1e-5, 1e-3, 1e-1],
    'l1_ratio': [0.15,0.5, 0.85],
    'tol' : [1e-5, 1e-6],
    'loss': ['log_loss']
}

grid_search = GridSearchCV(
    estimator=sgd,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=5,
    verbose=1,
    n_jobs=4
)

grid_search.fit(X_train_smote, y_train_smote)

# Best parameters and evaluation
print("Best Parameters:", grid_search.best_params_)
print("Best ROC-AUC:", grid_search.best_score_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best Parameters: {'alpha': 0.001, 'l1_ratio': 0.85, 'loss': 'log_loss', 'tol': 1e-05}
Best ROC-AUC: 0.805214371432065


In [44]:
for test_size in [0.2,0.5, 0.8]:
    X_train, X_test, y_train, y_test = train_test_split(X_n, y_n, test_size=test_size, random_state=42,shuffle=True)    
    
    sgd_opt = SGDClassifier(
        **grid_search.best_params_, 
        learning_rate='optimal',
        penalty='elasticnet',
        max_iter=1000,
        random_state=42
    )
    
    sgd_opt.fit(X_train, y_train)
    
    y_pred = sgd_opt.predict(X_test)
    y_pred_proba = sgd_opt.predict_proba(X_test)[:,1]
    
    print("Train="+str(round(1 - test_size, 2)))
    print(classification_report(y_test, y_pred, zero_division=0))
    print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")
    print(" ")
    print(" ")

Train=0.8
              precision    recall  f1-score   support

         0.0       0.60      0.59      0.60       451
         1.0       0.79      0.79      0.79       849

    accuracy                           0.72      1300
   macro avg       0.69      0.69      0.69      1300
weighted avg       0.72      0.72      0.72      1300

ROC-AUC: 0.7784
 
 
Train=0.5
              precision    recall  f1-score   support

         0.0       0.65      0.55      0.60      1157
         1.0       0.77      0.84      0.80      2092

    accuracy                           0.74      3249
   macro avg       0.71      0.70      0.70      3249
weighted avg       0.73      0.74      0.73      3249

ROC-AUC: 0.7907
 
 
Train=0.2
              precision    recall  f1-score   support

         0.0       0.64      0.62      0.63      1900
         1.0       0.79      0.80      0.79      3298

    accuracy                           0.73      5198
   macro avg       0.71      0.71      0.71      5198
weig

In [45]:
# save the model

dump(sgd_opt, "trained/sgd_opt_WINE.joblib")

['trained/sgd_opt_WINE.joblib']

## <b>Ensemble (Meta-model = SVM)</b>

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score

In [33]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X_n, y_n, test_size=0.2, random_state=42)    

# all pretrained models
base_models = [sgd_opt, rf_opt, nn_opt, xgb_opt]

# make meta features
meta_trainl = []
meta_testl = []

for model in base_models:
    if hasattr(model, "predict_proba"):
        meta_trainl.append(model.predict_proba(X_train)[:,1])
        meta_testl.append(model.predict_proba(X_test)[:,1])
    else:
        meta_trainl.append(model.predict(X_train).ravel())
        meta_testl.append(model.predict(X_test).ravel())    #NN doesnt have predict proba

meta_train = np.array(meta_trainl).T
meta_test = np.array(meta_testl).T

163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 469us/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 666us/step


In [48]:
svm_meta = SVC(kernel='rbf', probability=True, random_state=42)

param_grid = {
    'C' : [0.1, 1, 10],
    'gamma': [0.01, 0.1, 1]
}

grid_search = GridSearchCV(
    estimator=svm_meta,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=5,  # Inner cross-validation
    verbose=2,
    n_jobs=-1
)

# Fit the meta-model using GridSearchCV
grid_search.fit(meta_train, y_train)

# Best parameters and evaluation
print("Best Parameters:", grid_search.best_params_)
print("Best ROC-AUC:", grid_search.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best Parameters: {'C': 0.1, 'gamma': 0.01}
Best ROC-AUC: 0.8797707033252085


In [61]:
for test_size in [0.2,0.5, 0.8]:
    # split data
    X_train, X_test, y_train, y_test = train_test_split(X_n, y_n, test_size=test_size, random_state=42,shuffle=True)    

    meta_trainl=[]
    meta_testl=[]
    
    for model in base_models:
        if hasattr(model, "predict_proba"):
            meta_trainl.append(model.predict_proba(X_train)[:,1])
            meta_testl.append(model.predict_proba(X_test)[:,1])
        else:
            meta_trainl.append(model.predict(X_train).ravel())
            meta_testl.append(model.predict(X_test).ravel())    #NN doesnt have predict proba
    
    meta_train = np.array(meta_trainl).T
    meta_test = np.array(meta_testl).T
    
    svm_meta_opt = SVC(
        **grid_search.best_params_, 
        probability=True,
        random_state=42
    )
    
    svm_meta_opt.fit(meta_train, y_train)
    
    y_pred = svm_meta_opt.predict(meta_test)
    y_pred_proba = svm_meta_opt.predict_proba(meta_test)[:,1]
    
    print("Train="+str(round(1 - test_size, 2)))
    print(classification_report(y_test, y_pred, zero_division=0))
    print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")
    print(" ")
    print(" ")

163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Train=0.8
              precision    recall  f1-score   support

         0.0       0.63      0.67      0.65       451
         1.0       0.82      0.79      0.80       849

    accuracy                           0.75      1300
   macro avg       0.72      0.73      0.72      1300
weighted avg       0.75      0.75      0.75      1300

ROC-AUC: 0.8155
 
 
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Train=0.5
              precision    recall  f1-score   support

         0.0       0.67      0.68      0.67      1157
         1.0       0.82      0.82      0.82      2092

    accuracy                           0.77      3249
   macro avg       0.75      0.75      0.75      3249
weighted avg       0.77      0.77      0.77      3249

ROC-AUC: 0.8323
 
 
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Train=0.2
              precision

In [62]:
# save the model

dump(svm_meta_opt, "trained/svm_meta_opt_WINE.joblib")

['trained/svm_meta_opt_WINE.joblib']